In [2]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/language-detection/language-detection-data-v5.json.zip
# !unzip language-detection-data-v5.json.zip

In [3]:
import json

In [4]:
with open('language-detection-data-v5.json') as fopen:
    lang = json.load(fopen)

In [5]:
others = [text for no, text in enumerate(lang['text']) if lang['label'][no] not in ['zlm', 'ind']]
len(others)

381597

In [6]:
zlm = [text for no, text in enumerate(lang['text']) if lang['label'][no] == 'zlm']
len(zlm)

53692

In [7]:
zlm_words = set(' '.join(zlm).split())

In [8]:
ind = [text for no, text in enumerate(lang['text']) if lang['label'][no] == 'ind']
len(ind)

57327

In [9]:
ind_words = set(' '.join(ind).split())

In [10]:
len(zlm_words), len(ind_words)

(17286, 7614)

In [11]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/200k-english-malay/200k-english-malay.json

In [12]:
with open('200k-english-malay.json') as fopen:
    english_malay = json.load(fopen)
    
malays = set([i[1] for i in english_malay])
len(malays)

150125

In [13]:
# !wget https://dl.fbaipublicfiles.com/arrival/dictionaries/en-ms.txt

In [14]:
with open('en-ms.txt') as fopen:
    en_ms = list(filter(None, fopen.read().split('\n')))
    
en_ms = [i.split('\t') for i in en_ms]
en_ms = set([i[1] for i in en_ms if i[1] != i[0]])

In [15]:
malays = malays | en_ms

In [16]:
with open('id-wiki.txt') as fopen:
    id_wiki = fopen.read().split('\n')
    
len(id_wiki)

393429

In [17]:
import re
from unidecode import unidecode
import cleaning
from tqdm import tqdm

def preprocessing(string):
    string = unidecode(string)
    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub('[^A-Za-z ]+', ' ', unidecode(string))
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string.lower()

def loop(strings):
    for i in tqdm(range(len(strings))):
        strings[i] = preprocessing(strings[i])
    return strings

In [18]:
id_wiki = cleaning.multiprocessing(id_wiki, loop)

100%|██████████| 24589/24589 [00:32<00:00, 759.96it/s] 


In [19]:
id_wiki_words = set(' '.join(id_wiki).split())
len(id_wiki_words)

1435013

In [20]:
ind_words = ind_words | id_wiki_words

In [21]:
with open('../wikidump1-raw.json') as fopen:
    ms_wiki = json.load(fopen)
    
len(ms_wiki)

1748387

In [22]:
ms_wiki_words = set(' '.join(ms_wiki).split())
len(ms_wiki_words)

1893949

In [23]:
malays = malays | zlm_words | ms_wiki_words

In [24]:
with open('malays_word.json', 'w') as fopen:
    json.dump(list(malays), fopen)

In [24]:
intersected = malays.intersection(ind_words)

In [25]:
ms_wiki = cleaning.multiprocessing(ms_wiki, loop)

100%|██████████| 109274/109274 [00:02<00:00, 48206.83it/s]


In [26]:
ind_minus = ind_words - intersected
len(ind_minus)

1293392

In [27]:
for i in tqdm(range(len(ms_wiki))):
    ms_wiki[i] = ' '.join([w for w in ms_wiki[i].split() if w not in ind_minus])

100%|██████████| 1748387/1748387 [00:07<00:00, 223018.53it/s]


In [28]:
zlm[0]

'segi segi syaitan kawasan bahagian barat laut lautan utara kapal terbang kapal hilang misteri kawasan perairan segi laluan perairan sibuk dunia kapal kapal kawasan pelabuhan kepulauan kapal kapal mewah kapal terbang kawasan kehilangan dikaitkan aktiviti makhluk asing budaya popular dokumen kebanyakan disalah lapor samping penafian rasmi kebanyakan agensi kerajaan kehilangan ketara kes kehilangan kawasan siasatan kawasan segi kawasan segi sempadan segi pendapat bentuknya trapezium meliputi selat pulau timur teluk lautan lautan sempadan segi buku titik pantai lautan pulau kebanyakan misteri sempadan selatan selat lautan kawasan salah laluan kapal dunia kapal kapal pelabuhan kepulauan kapal pelancongan tersedia kapal alik kepulauan laluan kapal terbang arah selatan titik utara arus lautan arus teluk mengalir segi meninggalkan teluk arusnya knot peranan kehilangan ribut mengejut muncul musim panas musim luruh kadang kawasan gabungan laluan laut sibuk cuaca mustahil kapal karam ribut tengg

In [29]:
for i in tqdm(range(len(zlm))):
    zlm[i] = ' '.join([w for w in zlm[i].split() if w not in ind_minus])

100%|██████████| 53692/53692 [00:01<00:00, 41119.01it/s]


In [30]:
with open('ind_words.json', 'w') as fopen:
    json.dump(list(ind_minus), fopen)

In [31]:
with open('formal-language.json', 'w') as fopen:
    json.dump({'other': others,
              'malay': zlm + ms_wiki,
              'ind': ind + id_wiki}, fopen)